### Загрузка библиотек и данных

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords 
import re
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer 

import spacy

C:\Users\ament\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#nlp_en = spacy.load('en_core_web_sm')
nlp_ru = spacy.load("ru_core_news_sm")

In [3]:
#список дилеров
dealer = pd.read_csv('data/marketing_dealer.csv', sep=';', on_bad_lines='skip', engine='python')
#результат парсера
dealerprice = pd.read_csv('data/marketing_dealerprice.csv', sep=';', 
                                    on_bad_lines='skip', engine='python')
#данные заказчика
product = pd.read_csv('data/marketing_product.csv', sep=';', index_col=0)
#таблица соотвествия
productdealerkey = pd.read_csv('data/marketing_productdealerkey.csv', sep=';', 
                                         on_bad_lines='skip', engine='python')

In [4]:
for dataset in [dealer, dealerprice, product, productdealerkey]:
    display(dataset.head(5))

,id,name
0,1,Moi_vibor_WB
1,2,Akson
2,3,Bafus
3,5,Castorama
4,6,Cubatora


,id,product_key,price,product_url,product_name,date,dealer_id
0,2,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-11,2
1,3,546408,175.0,https://akson.ru//p/kontsentrat_prosept_multip...,"Концентрат Prosept Multipower для мытья полов,...",2023-07-11,2
2,4,546234,285.0,https://akson.ru//p/sredstvo_dlya_chistki_lyus...,Средство для чистки люстр Prosept Universal An...,2023-07-11,2
3,5,651258,362.0,https://akson.ru//p/udalitel_rzhavchiny_prosep...,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 0...",2023-07-11,2
4,6,546355,205.0,https://akson.ru//p/sredstvo_moyushchee_dlya_b...,Средство моющее для бани и сауны Prosept Multi...,2023-07-11,2


,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
0,245,008-1,4.680008e+12,Антисептик невымываемыйPROSEPT ULTRAконцентрат...,360.00,858.0,20.0,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,189522705.0,150033482.0,008-1,NaN
1,3,242-12,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,460.16,1075.0,NaN,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,NaN,NaN,NaN,NaN,NaN
2,443,0024-06 с,4.680008e+12,"Герметик акриловый цвет сосна, ф/п 600мл",307.00,644.0,25.0,Герметик акриловый для швов для деревянных дом...,"Герметик акриловый цвет сосна, ф/п 600мл",Герметик акриловый для швов для деревянных дом...,189522735.0,150126217.0,0024-06-с,NaN
3,147,305-2,4.610093e+12,Кондиционер для белья с ароматом королевского...,157.73,342.0,29.0,"Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...",339377922.0,150032962.0,305-2,NaN
4,502,0024-7 б,NaN,"Герметик акриловой цвет Белый, 7 кг",NaN,NaN,NaN,NaN,NaN,NaN,189522867.0,150126216.0,0024-7-б,NaN


,id,key,dealer_id,product_id
0,1,546227,2,12
1,2,651265,2,106
2,3,546257,2,200
3,4,546408,2,38
4,5,651258,2,403


### Формирование проверочного датасета

In [5]:
df = pd.merge(dealerprice, productdealerkey.rename(columns={'key': 'product_key'}), on = 'product_key', how='left')
df = pd.merge(df, product.rename(columns={'id': 'product_id'}), on = 'product_id', how='left')
df = df[['product_name', 'product_key', 'product_id', 'name']]
df = df.drop_duplicates().dropna()
df = df.set_index(df['product_key'])
df

,product_name,product_key,product_id,name
product_key,,,,
546227,Средство универсальное Prosept Universal Spray...,546227,12.0,Универсальное моющее и чистящее средствоUniver...
546408,"Концентрат Prosept Multipower для мытья полов,...",546408,38.0,Концентрат для мытья половMULTIPOWER с аромато...
546234,Средство для чистки люстр Prosept Universal An...,546234,18.0,Средство для чистки люстр Universal Anti-dust ...
651258,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 0...",651258,403.0,Удалитель ржавчины RUST REMOVER готовый состав...
546355,Средство моющее для бани и сауны Prosept Multi...,546355,39.0,Моющее средство для бани и сауныMultipower Woo...
...,...,...,...,...
44231991,Жидкое моющее средство для стирки Crystal конц...,44231991,150.0,Жидкий моющий концентрат для стирки белья с а...
1462340,Средство для мытья плитки керамогранита Prosep...,1462340,40.0,Средство для мытья плитки и керамогранитаMulti...
40019201,CEMENT CLEANER удалитель цемента,40019201,400.0,Удалитель цемента CEMENT CLEANER готовый соста...


In [6]:
SPACES = r'(?<=[а-яА-Я])(?=[a-zA-Z])|(?<=[a-zA-Z])(?=[а-яА-Я])'
DUP_SPACES = r'([ ])\1+'

In [7]:
def clean_string(input_string):
    input_string = re.sub(SPACES, ' ', input_string.lower()) # нижний регистр, обработка пропущенных пробелов
    input_string = input_string.replace('prosept', '') # убираем название фирмы
    input_string = input_string.replace('просепт', '')
    input_string = input_string.replace(r'[^\w\s]', ' ') 
    input_string = input_string.replace('.', ' ')
    input_string = input_string.replace(')', ' ')
    input_string = input_string.replace('(', ' ')
    input_string = input_string.replace(',', ' ')
    input_string = input_string.replace(':', ' ')
    input_string = input_string.replace('-', ' ')
    input_string = re.sub(DUP_SPACES, r'\1', input_string) # обработка двойных пробелов
    input_string = ' '.join(re.split('(\d+)', input_string))
    return input_string

In [8]:
# применение:
df['name'] = df['name'].apply(clean_string)
df['product_name'] = df['product_name'].apply(clean_string)
product['name'] = product['name'].astype('str').apply(clean_string)
df.head()

,product_name,product_key,product_id,name
product_key,,,,
546227,средство универсальное universal spray 500 мл,546227,12.0,универсальное моющее и чистящее средство unive...
546408,концентрат multipower для мытья полов цитрус 1 л,546408,38.0,концентрат для мытья полов multipower с аромат...
546234,средство для чистки люстр universal anti dust ...,546234,18.0,средство для чистки люстр universal anti dust ...
651258,удалитель ржавчины rust remover 0 5 л 023 ...,651258,403.0,удалитель ржавчины rust remover готовый состав...
546355,средство моющее для бани и сауны multipower wo...,546355,39.0,моющее средство для бани и сауны multipower wo...


In [9]:
def lemm (text):
    
    lemmed = nlp_ru(text)   
    prepaired = " ".join([token.lemma_ for token in lemmed])
       
    return prepaired

Wall time: 0 ns


In [10]:
df['name'] = df['name'].apply(lemm)

In [11]:
nltk.download('stopwords') 
stopwords_en = set(stopwords.words('english')) 
stopwords_ru = set(stopwords.words('russian')) 
stopwords = stopwords_en | stopwords_ru

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ament\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
#этого процесса на практике не будет, т.к. новые описания добавляются скорее десятками штук, а не 1600 сразу
df['product_name'] = df['product_name'].apply(lemm)

In [13]:
product['name'] = product['name'].apply(lemm)

In [14]:
def kneighbors_search(line,nn):
    vectorized_line = count_tf_idf.transform([line])
    result = product.iloc[model.kneighbors(vectorized_line, 
                                           n_neighbors=nn, return_distance=False)[0]]['id'].values
    return result

In [15]:
def accuracy(df, result):
    success = 0
    failure = []
    for i in range (len(df)):
        if df['product_id'].astype('int')[i] in result[i]:
            success+=1
        else: failure.append(i)
    return success/len(df), failure

In [16]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords, decode_error='ignore', analyzer='char_wb', ngram_range=(1,2))
assort = product['name'].apply(clean_string).reindex()
vectorized = count_tf_idf.fit_transform(assort)
model = NearestNeighbors(metric='cosine')
model.fit(vectorized)

NearestNeighbors(metric='cosine')

Метрика для n = 5 

In [17]:
%%time
result = df['product_name'].apply(kneighbors_search, nn=5)
result
ac_5 = accuracy(df,result)
ac_5[0]

Wall time: 4.53 s


0.9166173861620343

Метрика для разных n.

In [18]:
nn=[1, 3, 5, 10, 15, 20, 25]
print('tf_idf')
for n in nn:
    result = df['product_name'].apply(kneighbors_search, nn=n)
    print(f'Accuracy@{n}:', accuracy(df,result)[0])

tf_idf
Accuracy@1: 0.6966292134831461
Accuracy@3: 0.8769958604376109
Accuracy@5: 0.9166173861620343
Accuracy@10: 0.9603784742755765
Accuracy@15: 0.9704316972205795
Accuracy@20: 0.9793021880544057
Accuracy@25: 0.9816676522767593


Посмотрим на записи, к которым не удалось подобрать товар из базы.

In [19]:
df.iloc[ac_5[1]]

,product_name,product_key,product_id,name
product_key,,,,
831854,антисептик eco ultra невымываемый коричневый г...,831854,275.0,антисептик eco ultra коричневый / 5 л
831853,антисептик eco ultra невымываемый коричневый г...,831853,276.0,антисептик eco ultra коричневый / 10 л
651268,"концентрат для мытьё пол и стена multipower "" ...",651268,44.0,"средство для мытьё пол и стена multipower "" по..."
200671203,bio lasur антисептик лессирующий защитно дек...,200671203,332.0,антисептик лессирующий bio lasur / белый люкс ...
200692763,bio lasur антисептик лессирующий защитно дек...,200692763,341.0,антисептик лессирующий bio lasur / дуб / 2 ...
...,...,...,...,...
529985141,строительный антисептик 5 6 кг,529985141,264.0,антисептик для внутренний работа sauna концент...
530305153,антисептик невымываемый eco ultra коричневый ...,530305153,276.0,антисептик eco ultra коричневый / 10 л
531730388,средство для удаление ржавчина 1 шт,531730388,405.0,удалитель ржавчина rust remover концентрат 1...
